# Chargement de la base de données

Remarques avant de construire predictions71:
Il faut lancer 11 fois ce code avec 10 fois 10000 images et une fois le reste des images à passer. A chaque fois il faut ré-écrire dan sun nouveau nom de csv pour qu'il puisse lire le précédent csv. Il faut aussi modifier les variables lb et up et leur donner l'encadrement des images sur lesquelles on veut faire un predict. Ensuite il ne faut pas oublier de changer la première ligne du predict lorsqu'on applique la fonction de predict au dataset.

In [104]:
import os
from tensorflow import keras
from tensorflow.keras import layers

In [105]:
path = "/home/acarlier/Desktop/"

In [106]:
path_directory = path + "Cropped_BD_71/"
print(path_directory)

/home/acarlier/Desktop/Cropped_BD_71/


In [107]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory


IMG_SIZE = 224
train_ds = keras.utils.image_dataset_from_directory(
    directory=path_directory,
    labels='inferred',
    label_mode='categorical',
    shuffle = False,
    batch_size=16,
    image_size=(IMG_SIZE, IMG_SIZE))

Found 105597 files belonging to 71 classes.


In [108]:
class_names = train_ds.class_names
print(class_names)
nb_classes = len(class_names)
print(nb_classes)

['Amegilla quadrifasciata', 'Andrena agilissima', 'Andrena bicolor', 'Andrena cineraria', 'Andrena clarkella', 'Andrena denticulata', 'Andrena flavipes', 'Andrena florea', 'Andrena fulva', 'Andrena gravida', 'Andrena haemorrhoa', 'Andrena hattorfiana', 'Andrena nigroaenea', 'Andrena nitida', 'Andrena nycthemera', 'Andrena thoracica', 'Andrena vaga', 'Andrena ventralis', 'Anthidiellum strigatum', 'Anthidium florentinum', 'Anthidium manicatum', 'Anthidium oblongatum', 'Anthidium septemspinosum', 'Anthophora bimaculata', 'Anthophora furcata', 'Anthophora plumipes', 'Apis mellifera', 'Bombus argillaceus', 'Bombus bohemicus', 'Bombus campestris', 'Bombus hortorum', 'Bombus humilis', 'Bombus hypnorum', 'Bombus lapidarius', 'Bombus lucorum', 'Bombus muscorum', 'Bombus pascuorum', 'Bombus pratorum', 'Bombus ruderatus', 'Bombus rupestris', 'Bombus sylvarum', 'Bombus terrestris lusitanicus', 'Bombus vestalis', 'Ceratina cucurbitina', 'Chelostoma florisomne', 'Colletes cunicularius', 'Colletes he

# Chargement du modèle

In [109]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
import tensorflow as tf

In [110]:
# Création de l'architecture du modèle à utiliser
conv_base = keras.applications.resnet.ResNet101(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling=None,
    classes=nb_classes,
)

model = keras.Sequential(
    [
        conv_base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(nb_classes, kernel_regularizer=regularizers.L2(1e-4), activation='softmax')
    ]
)

model.load_weights("/home/acarlier/Desktop/poids/best_model_hierarchicaloss") 

model.compile(optimizers.SGD(learning_rate=1e-3, momentum=0.9), loss='categorical_crossentropy', metrics=['categorical_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [111]:
import numpy as np
import time

start_time = time.time()
x = np.zeros((10000, IMG_SIZE, IMG_SIZE, 3))#le chiffre est le nombre d'images dans le dossier sur lesquelles on va faire la prediction dans un deuxième temps
y = np.zeros((10000, nb_classes))

ind_data = 0
lb = 40000
up = 50000
for bx, by in train_ds.as_numpy_iterator():
    if (ind_data < lb):
        ind_data += bx.shape[0]
    elif (ind_data >= lb and ind_data < up):
        x[ind_data-lb:ind_data-lb+bx.shape[0]] = bx
        y[ind_data-lb:ind_data-lb+bx.shape[0]] = by
        ind_data += bx.shape[0]
    elif (ind_data >= up):
        break
print("--- %s seconds ---" % (time.time() - start_time))
x = tf.keras.applications.resnet.preprocess_input(x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 14.65914011001587 seconds ---
--- 19.42300033569336 seconds ---


In [112]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 2


In [113]:
y_pred = model.predict(x)

In [ ]:
import tensorflow
from tensorflow.keras.metrics import Accuracy

In [ ]:
test_accuracy = Accuracy()

prediction = tensorflow.argmax(y_pred, axis=1, output_type=tensorflow.int32)
print(prediction)
test_accuracy(prediction, np.argmax(y, axis=1))

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

tf.Tensor([45 26 45 ... 23 23 42], shape=(10000,), dtype=int32)
Test set accuracy: 32.700%


In [ ]:
print(y_pred)
print(y_pred.shape) #pk y_pred juste de 3979?
print(y)

[[1.3342589e-03 3.2857547e-03 9.2629658e-04 ... 6.9220673e-04
  1.0784476e-03 2.8258466e-04]
 [4.5362282e-05 1.3329812e-05 1.2545778e-03 ... 1.3697407e-05
  5.9859944e-05 4.7884187e-06]
 [2.0404768e-03 6.7935442e-04 1.1276222e-03 ... 8.4629188e-05
  4.2118525e-04 4.2743683e-05]
 ...
 [8.7254338e-02 2.4993487e-03 7.9852447e-04 ... 4.7648119e-04
  1.3419227e-03 9.2367540e-05]
 [5.0278031e-03 9.1842861e-07 1.5672787e-05 ... 3.3703942e-05
  1.8009789e-05 1.5429121e-04]
 [7.7372817e-03 2.6081262e-02 3.3990212e-03 ... 8.2308194e-03
  6.6178478e-02 1.4906117e-02]]
(10000, 71)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
files = train_ds.file_paths

In [ ]:
files[24000]

'/home/acarlier/Desktop/Cropped_BD_71/Apis mellifera/Apis mellifera19629.jpg'

# Création d'un DataFrame pour récupérer les prédictions

In [ ]:
import pandas as pd
import os

In [ ]:
# noms des fichiers et noms des classes
file_paths = train_ds.file_paths
class_names = train_ds.class_names

In [ ]:
def save_pred_df(df, y_pred, y, starting_row=0):
    """
    Sauvegarder les informations d'une predictions dans un dataframe.
    
    Arguments :
    - df : le dataframe où on ajoute les informations : doit contenir
    species_name, species_num, ...
    - y_pred : la prédiction
    """
    
    nb_rows = y_pred.shape[0] # nombre de lignes de la matrice y_pred (.shape renvoie dimensiosn matrice)
    
    for i in range(nb_rows):
        # nom de l'espèce
        class_num = np.argmax(y[i])
        class_name = class_names[class_num]

        # nom de l'image et path de l'image
        image_name = os.path.basename(files[i + starting_row]) 
        relative_image_path = "Cropped_BD_71/" + class_name + "/" + image_name

        # proba associée à la ground truth
        probability = y_pred[i, class_num]
        predicted_species_num = np.argmax(y_pred[i])
        predicted_species_name = class_names[predicted_species_num]

        # Ajout des données au DataFrame
        row_to_add = {"species_name" : class_name,
                      "species_num" : class_num,
                      "predicted_species_name" : predicted_species_name, 
                      "predicted_species_num" : predicted_species_num,
                      "good_pred_probability" : probability,
                      "img_name" : image_name, 
                      "img_path" : relative_image_path}
        df = df.append(row_to_add, ignore_index=True)
        
    return df

In [ ]:
dic = {"species_name" : [], "species_num" : [], "predicted_species_name" : [], "predicted_species_num" : [], "good_pred_probability" : [], "img_name" : [], "img_path" : []}
df = pd.DataFrame(dic)
display(df)

,species_name,species_num,predicted_species_name,predicted_species_num,good_pred_probability,img_name,img_path


In [ ]:
df = pd.read_csv( "/home/acarlier/Desktop/predictions74.csv")#à changer avec le précédent fichier
display(df)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,species_name,species_num,predicted_species_name,predicted_species_num,good_pred_probability,img_name,img_path
0,0,0.0,0.0,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.803115,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,Cropped_BD_71/Amegilla quadrifasciata/2019-08-...
1,1,1.0,1.0,Amegilla quadrifasciata,0.0,Anthophora plumipes,25.0,0.002693,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,Cropped_BD_71/Amegilla quadrifasciata/2019-08-...
2,2,2.0,2.0,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.993149,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,Cropped_BD_71/Amegilla quadrifasciata/2019-08-...
3,3,3.0,3.0,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.998210,Amegilla quadrifasciata10653.jpg,Cropped_BD_71/Amegilla quadrifasciata/Amegilla...
4,4,4.0,4.0,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.999889,Amegilla quadrifasciata12804.jpg,Cropped_BD_71/Amegilla quadrifasciata/Amegilla...
...,...,...,...,...,...,...,...,...,...,...
29995,29995,NaN,NaN,Apis mellifera,26.0,Apis mellifera,26.0,0.916228,Apis mellifera32022.jpeg,Cropped_BD_71/Apis mellifera/Apis mellifera320...
29996,29996,NaN,NaN,Apis mellifera,26.0,Osmia bicolor,62.0,0.027007,Apis mellifera32024.jpeg,Cropped_BD_71/Apis mellifera/Apis mellifera320...
29997,29997,NaN,NaN,Apis mellifera,26.0,Anthophora plumipes,25.0,0.011482,Apis mellifera32025.jpg,Cropped_BD_71/Apis mellifera/Apis mellifera320...
29998,29998,NaN,NaN,Apis mellifera,26.0,Apis mellifera,26.0,0.925262,Apis mellifera32028.jpg,Cropped_BD_71/Apis mellifera/Apis mellifera320...


In [ ]:
# df.drop(columns='Unnamed: 0', inplace=True)
# display(df)

In [ ]:
# print(files[111978])

In [ ]:
df = save_pred_df(df, y_pred, y, starting_row = 40000) #bien changer starting raw et mettre la valeur de lb
display(df)

/tmp/ipykernel_30654/3383503863.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_to_add, ignore_index=True)
/tmp/ipykernel_30654/3383503863.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_to_add, ignore_index=True)
/tmp/ipykernel_30654/3383503863.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_to_add, ignore_index=True)
/tmp/ipykernel_30654/3383503863.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_to_add, ignore_index=True)
/tmp/ipykernel_30654/3383503863.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pan

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,species_name,species_num,predicted_species_name,predicted_species_num,good_pred_probability,img_name,img_path
0,0.0,0.0,0.0,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.803115,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,Cropped_BD_71/Amegilla quadrifasciata/2019-08-...
1,1.0,1.0,1.0,Amegilla quadrifasciata,0.0,Anthophora plumipes,25.0,0.002693,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,Cropped_BD_71/Amegilla quadrifasciata/2019-08-...
2,2.0,2.0,2.0,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.993149,2019-08-01 Luminy Hymenoptera 10 a 12 mm Apida...,Cropped_BD_71/Amegilla quadrifasciata/2019-08-...
3,3.0,3.0,3.0,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.998210,Amegilla quadrifasciata10653.jpg,Cropped_BD_71/Amegilla quadrifasciata/Amegilla...
4,4.0,4.0,4.0,Amegilla quadrifasciata,0.0,Amegilla quadrifasciata,0.0,0.999889,Amegilla quadrifasciata12804.jpg,Cropped_BD_71/Amegilla quadrifasciata/Amegilla...
...,...,...,...,...,...,...,...,...,...,...
39995,NaN,NaN,NaN,Apis mellifera,26.0,Apis mellifera,26.0,0.958697,Apis mellifera5721.jpg,Cropped_BD_71/Apis mellifera/Apis mellifera572...
39996,NaN,NaN,NaN,Apis mellifera,26.0,Andrena ventralis,17.0,0.000507,Apis mellifera57210.jpg,Cropped_BD_71/Apis mellifera/Apis mellifera572...
39997,NaN,NaN,NaN,Apis mellifera,26.0,Anthophora bimaculata,23.0,0.025020,Apis mellifera57215.jpeg,Cropped_BD_71/Apis mellifera/Apis mellifera572...
39998,NaN,NaN,NaN,Apis mellifera,26.0,Anthophora bimaculata,23.0,0.013480,Apis mellifera57216.jpg,Cropped_BD_71/Apis mellifera/Apis mellifera572...


In [ ]:
# for i in range(12000, 24000):
#    image_name = os.path.basename(files[i])
#    class_name = "Apis mellifera"
#    relative_image_path = "BD_71/" + class_name + "/" + image_name
#    df.at[i, "img_name"] = image_name
#    df.at[i, "img_path"] = relative_image_path

In [ ]:
#path = os.getcwd()
#df.drop(columns="Unnamed: 0", inplace=True)
#display(df)
df.to_csv("/home/acarlier/Desktop" + "/predictions75.csv")

In [ ]:
# img_path = df["img_path"]
# print(img_path)
# #on veut remplacer tous les BD par cropped_BD
# for i in range(len(img_path)):
#     try:
#         img_path[i] = img_path[i].replace("Cropped_Cropped_BD_71", "Cropped_BD_71")
#     except:
#         img_path[i] = img_path[i].replace("BD_71", "Cropped_BD_71")
#     print(img_path[i])
# df["img_path"] = img_path
# display(df)
# df.to_csv("/home/acarlier/Desktop" + "/predictions78_bis.csv")